<a href="https://colab.research.google.com/github/fransindi/recomendacion_amazon/blob/master/Modelo_ML_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proceso ML

### Instalamos las librerias necesarias.

In [ ]:
!pip install scikit-surprise
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163341 sha256=fb75652578ceab1879a191503e9602594813414da7457fb0ce6aa3c9755e2240
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


### importamos los datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importamos las librerias

In [ ]:
#Importar librerías necesarias
import csv
import pandas as pd
import json
import gzip
import ast
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import nltk
nltk.download('stopwords')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### creamos funcion para separar y extraer las palabras relevantes

In [ ]:
def extract_keywords(text):
    # Tokenizar el texto en palabras
    words = word_tokenize(text)

    # Obtener las stopwords en español
    stop_words = set(stopwords.words('english'))

    # Filtrar las palabras clave quitando las stopwords y las que tienen menos de 3 caracteres
    keywords = [word.lower() for word in words if word.lower() not in stop_words and len(word) > 2]

    return keywords

### creamos listas con los datos

In [ ]:
archivos_review = ['/content/drive/MyDrive/Colab Notebooks/Amazon sample clean datasets/All_Beauty_Processed.json', '/content/drive/MyDrive/Colab Notebooks/Amazon sample clean datasets/Digital_Music_Processed.json','/content/drive/MyDrive/Colab Notebooks/Amazon sample clean datasets/Pet_Supplies_Processed.json','/content/drive/MyDrive/Colab Notebooks/Amazon sample clean datasets/Toys_and_Games_Processed.json']
archivos_meta = ['/content/drive/MyDrive/Colab Notebooks/Amazon sample clean datasets/All_Beauty_meta_Processed.json', '/content/drive/MyDrive/Colab Notebooks/Amazon sample clean datasets/Digital_Music_meta_Processed.json','/content/drive/MyDrive/Colab Notebooks/Amazon sample clean datasets/Pet_Supplies_meta_Processed.json','/content/drive/MyDrive/Colab Notebooks/Amazon sample clean datasets/Toys_and_Games_meta_Processed.json']

### creamos la funcion 'proceso_ml' para que ingresando los datos automaticamente haga el proceso de separacion de palabras con NLTK y usamos SVD para entrenar el modelo.

In [ ]:
def proceso_ml(json_review, json_meta):
    with open(json_review, 'r') as archivo_json:
    # Lee líneas del archivo y carga cada línea como un objeto JSON
        objetos_json = [json.loads(line) for line in archivo_json]

    # Ahora, objetos_json es una lista de diccionarios
    # Creamos Dataframe de reviews
    df = pd.DataFrame(objetos_json)

    # Abre el archivo JSON
    with open(json_meta, 'r') as archivo_json:
    # Lee líneas del archivo y carga cada línea como un objeto JSON
        objetos_json = [json.loads(line) for line in archivo_json]

    # Ahora, objetos_json es una lista de diccionarios
    dfmeta = pd.DataFrame(objetos_json)

    df_completo = df.merge(dfmeta, on="CodigoProducto")

    # Crear un objeto Reader para cargar los datos
    reader = Reader(rating_scale=(1, 5))

    # Cargar los datos desde el DataFrame completo
    data = Dataset.load_from_df(df_completo[["CodigoUsuario", "CodigoProducto", "Overall"]], reader)

    # Dividir los datos en conjuntos de entrenamiento y prueba (80% para entrenamiento, 20% para prueba)
    trainset, testset = train_test_split(data, test_size=0.2)

    # Elegir un algoritmo de recomendación (en este caso, SVD)
    algo = SVD()

    # Entrenar el modelo en el conjunto de entrenamiento
    algo.fit(trainset)

    # Obtener las predicciones de calificaciones para el conjunto de prueba
    predictions = algo.test(testset)

    # Evaluar el rendimiento del modelo calculando el RMSE
    rmse = accuracy.rmse(predictions)

    product_keywords = dfmeta['NombreProducto'].apply(extract_keywords)
    all_keywords = [keyword for keywords in product_keywords for keyword in keywords]
    keyword_counts = Counter(all_keywords)

    top_keywords = [keyword for keyword, count in keyword_counts.most_common(1000)]

        # Define tus 10 palabras clave
    palabras_clave = top_keywords

    # Crea un DataFrame para almacenar las recomendaciones
    recomendaciones_df = pd.DataFrame(columns=["PalabraClave", "Recomendaciones"])

    # Crear un objeto Reader para cargar los datos
    reader = Reader(rating_scale=(1, 5))

    # Cargar los datos desde el DataFrame completo
    data = Dataset.load_from_df(df_completo[["CodigoUsuario", "CodigoProducto", "Overall"]], reader)

    # Elegir un algoritmo de recomendación (en este caso, SVD)
    algo = SVD()

    # Entrenar el modelo en el conjunto de datos completo
    trainset = data.build_full_trainset()
    algo.fit(trainset)

    # Crear un diccionario para almacenar las recomendaciones por palabra clave
    recomendaciones_por_palabra = {}

    # Iterar a través de las palabras clave
    for palabra_clave in palabras_clave:
        recomendaciones = []

        # Crear una expresión regular para buscar la palabra clave en el nombre o descripción del producto
        expresion_regular = re.compile(f".*{re.escape(palabra_clave)}.*", flags=re.IGNORECASE)

        # Buscar productos que coincidan con la palabra clave
        productos_coincidentes = dfmeta[dfmeta['NombreProducto'].apply(lambda x: bool(expresion_regular.match(x)))]

        for codigo_producto in productos_coincidentes['CodigoProducto']:
            if codigo_producto not in df_completo['CodigoProducto'].values:
                recomendaciones.append(codigo_producto)

        # Obtener los nombres y precios de los productos correspondientes a los códigos
        productos_recomendados = dfmeta[dfmeta['CodigoProducto'].isin(recomendaciones)][['NombreProducto', 'PrecioProducto']]
        recomendaciones_por_palabra[palabra_clave] = productos_recomendados[:5].to_dict(orient='records')

    # Agregar las recomendaciones al DataFrame
    for palabra_clave, recomendaciones in recomendaciones_por_palabra.items():
        recomendaciones_df = recomendaciones_df.append({"PalabraClave": palabra_clave, "Recomendaciones": recomendaciones}, ignore_index=True)

    # Guardar las recomendaciones en formato JSON
    recomendaciones_df.to_json("recomendaciones.json", orient="records")

    return recomendaciones_df

### creamos los dataframes

In [ ]:
df_all_beauty = proceso_ml(archivos_review[0], archivos_meta[0])
df_digital_music = proceso_ml(archivos_review[1], archivos_meta[1])
df_pets = proceso_ml(archivos_review[2], archivos_meta[2])
df_toys = proceso_ml(archivos_review[3], archivos_meta[3])

RMSE: 1.3675


### Extraemos las tablas creadas

In [ ]:
df_all_beauty.to_csv('ml_all_beauty.csv')
df_digital_music.to_csv('ml_digital_music.csv')
df_pets.to_csv('ml_pet.csv')
df_toys.to_csv('ml_toys.csv')